In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from datetime import timedelta
import warnings

warnings.filterwarnings("ignore")

df = pd.read_csv('Lloyd_TV_Demand_Dataset.csv')
print("Columns:", df.columns.tolist())

Columns: ['Customer_ID', 'Customer_Name', 'Model', 'Screen_Size_Inch', 'Region', 'Purchase_Channel', 'Payment_Method', 'Warranty_Years', 'Purchase_Date', 'Unit_Price', 'Quantity', 'Discount_Percent', 'Promo_Applied', 'Final_Price', 'Repeat_Buyer', 'Days_Since_Last_Purchase', 'Return_Flag', 'Rating', 'CLV_Segment']


In [3]:
def parseable_info(series):
    try:
        parsed = pd.to_datetime(series, errors='coerce')
        non_na = parsed.notna().mean()
        years = parsed.dropna().dt.year
        if len(years)>0:
            miny, maxy = years.min(), years.max()
        else:
            miny, maxy = None, None
        return non_na, miny, maxy
    except Exception:
        return 0.0, None, None

In [5]:
obj_cols = df.select_dtypes(include=['object']).columns.tolist()
scores = []
for c in obj_cols:
    non_na, miny, maxy = parseable_info(df[c])
    scores.append((c, non_na, miny, maxy))
scores_sorted = sorted(scores, key=lambda x: x[1], reverse=True)

In [7]:
date_col = None
for c, non_na, miny, maxy in scores_sorted:
    if non_na > 0.3 and miny and 2000 <= miny <= 2035:
        date_col = c
        break
if date_col is None and scores_sorted:
    date_col = scores_sorted[0][0]

if date_col is None:
    raise ValueError("Could not confidently detect a date column. Please ensure CSV has a date column.")

In [9]:
if 'Quantity' in df.columns:
    target_col = 'Quantity'
else:
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if not numeric_cols:
        raise ValueError("No numeric columns found for target.")
    target_col = max(numeric_cols, key=lambda c: df[c].var())

print("Using date column:", date_col)
print("Using target column:", target_col)

Using date column: Purchase_Date
Using target column: Quantity


In [11]:
df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
df = df.dropna(subset=[date_col]).sort_values(date_col).reset_index(drop=True)

df_daily = df.set_index(date_col)[target_col].resample('D').sum().to_frame().rename(columns={target_col:'demand'})
df_daily['demand'] = df_daily['demand'].interpolate(method='time').ffill().bfill()

In [13]:
df_feat = df_daily.copy()
df_feat['date'] = df_feat.index
df_feat['day'] = df_feat['date'].dt.day
df_feat['month'] = df_feat['date'].dt.month
df_feat['year'] = df_feat['date'].dt.year
df_feat['weekday'] = df_feat['date'].dt.weekday
df_feat['is_weekend'] = df_feat['weekday'] >= 5
for lag in [1,7,14,30]:
    df_feat[f'lag_{lag}'] = df_feat['demand'].shift(lag)
df_feat['r7'] = df_feat['demand'].shift(1).rolling(7, min_periods=1).mean()
df_feat['r30'] = df_feat['demand'].shift(1).rolling(30, min_periods=1).mean()
df_feat = df_feat.dropna().copy()

features = ['day','month','weekday','is_weekend','lag_1','lag_7','lag_14','lag_30','r7','r30']

In [15]:
test_days = min(90, int(len(df_feat)*0.2))
if test_days < 1:
    test_days = max(1, int(len(df_feat)*0.2))
train = df_feat.iloc[:-test_days] if test_days < len(df_feat) else df_feat.iloc[:-1]
test = df_feat.iloc[-test_days:]

X_train = train[features]; y_train = train['demand']
X_test = test[features]; y_test = test['demand']

In [17]:
model = RandomForestRegressor(n_estimators=150, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)
pred_test = model.predict(X_test)

In [19]:
mae = mean_absolute_error(y_test, pred_test)
rmse = np.sqrt(mean_squared_error(y_test, pred_test))  # manual RMSE
r2 = r2_score(y_test, pred_test)

mape = np.mean(np.abs((y_test - pred_test) / (y_test.replace(0, np.nan)))) * 100
mape = np.nan_to_num(mape, nan=100.0)

accuracy_percent = max(0.0, 100.0 - mape)

print(f"MAE: {mae:.3f}, RMSE: {rmse:.3f}, R2: {r2:.4f}, MAPE: {mape:.2f}% -> Accuracy ≈ {accuracy_percent:.2f}%")

MAE: 0.745, RMSE: 0.910, R2: -0.0778, MAPE: 35.52% -> Accuracy ≈ 64.48%


In [21]:
last_date = df_feat['date'].max()
start_date = last_date + timedelta(days=1)
end_date = start_date + pd.DateOffset(months=6) - timedelta(days=1)
future_dates = pd.date_range(start=start_date, end=end_date, freq='D')

history = df_daily['demand'].copy()
future_preds = []
for current_date in future_dates:
    day = current_date.day
    month = current_date.month
    weekday = current_date.weekday()
    is_weekend = int(weekday >= 5)
    def safe_lag(d, lag):
        idx = d - timedelta(days=lag)
        if idx in history.index:
            return history.loc[idx]
        else:
            return history.iloc[-1]
    lag_1 = safe_lag(current_date,1)
    lag_7 = safe_lag(current_date,7)
    lag_14 = safe_lag(current_date,14)
    lag_30 = safe_lag(current_date,30)
    r7 = history.reindex(pd.date_range(end=current_date - timedelta(days=1), periods=7, freq='D')).mean()
    r30 = history.reindex(pd.date_range(end=current_date - timedelta(days=1), periods=30, freq='D')).mean()
    if pd.isna(r7): r7 = history.mean()
    if pd.isna(r30): r30 = history.mean()
    feat = np.array([[day, month, weekday, is_weekend, lag_1, lag_7, lag_14, lag_30, r7, r30]])
    pred = model.predict(feat)[0]
    history.loc[current_date] = pred
    future_preds.append({'date': current_date, 'predicted_demand': pred})

future_df = pd.DataFrame(future_preds).set_index('date')
def ordinal(n):
    if 10 <= n % 100 <= 20:
        suf = 'th'
    else:
        suf = {1:'st',2:'nd',3:'rd'}.get(n%10,'th')
    return str(n) + suf
future_df['day_label'] = [f"{ordinal(d.day)} {d.strftime('%B %Y')}" for d in future_df.index]
future_df['predicted_demand_rounded'] = future_df['predicted_demand'].round(2)

In [22]:
summary = []
for m, group in future_df.groupby(future_df.index.to_period('M')):
    future_avg = group['predicted_demand'].mean()
    prev_year_period = (m - 12)
    prev_same = None
    try:
        prev_same_idx = df_daily.loc[ (df_daily.index.to_period('M') == prev_year_period) , 'demand']
        if len(prev_same_idx) > 0:
            prev_same = prev_same_idx.mean()
    except Exception:
        prev_same = None
    if prev_same is not None and not np.isnan(prev_same):
        pct_change = (future_avg - prev_same) / prev_same * 100.0 if prev_same!=0 else np.nan
        direction = "increasing" if pct_change>0 else "decreasing" if pct_change<0 else "stable"
        basis = f"vs same month last year ({prev_year_period.start_time.date()})"
    else:
        recent_hist = df_daily.tail(30)['demand'].mean()
        pct_change = (future_avg - recent_hist) / recent_hist * 100.0 if recent_hist!=0 else np.nan
        direction = "increasing" if pct_change>0 else "decreasing" if pct_change<0 else "stable"
        basis = "vs recent historical month (last 30 days)"
    summary.append({
        'month': m.strftime('%B %Y'),
        'future_avg_demand': round(future_avg,2),
        'comparison_basis': basis,
        'pct_change': round(pct_change,2) if not pd.isna(pct_change) else None,
        'direction': direction
    })
summary_df = pd.DataFrame(summary)

In [25]:
out_path = r"C:\Users\ADITYA KUSHWAHA\Desktop\AIM\demand_forecast_next_6_months.csv"
future_df.reset_index().to_csv(out_path, index=False)
print("Forecast saved to", out_path)

Forecast saved to C:\Users\ADITYA KUSHWAHA\Desktop\AIM\demand_forecast_next_6_months.csv


In [27]:
print("\nSample future predictions (first 10):")
print(future_df[['day_label','predicted_demand_rounded']].head(10).to_string())
print("\nMonth summary:")
print(summary_df.to_string(index=False))


Sample future predictions (first 10):
                     day_label  predicted_demand_rounded
date                                                    
2025-08-31    31st August 2025                      0.83
2025-09-01  1st September 2025                      0.96
2025-09-02  2nd September 2025                      0.89
2025-09-03  3rd September 2025                      0.73
2025-09-04  4th September 2025                      1.04
2025-09-05  5th September 2025                      0.75
2025-09-06  6th September 2025                      0.70
2025-09-07  7th September 2025                      0.71
2025-09-08  8th September 2025                      1.51
2025-09-09  9th September 2025                      0.95

Month summary:
         month  future_avg_demand                     comparison_basis  pct_change  direction
   August 2025               0.83 vs same month last year (2024-08-01)      -10.92 decreasing
September 2025               0.91 vs same month last year (2024-09-01)   